<a href="https://colab.research.google.com/github/gdabrow/Investments/blob/main/stockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Information on companies listed on stock exchanges obtained from:**
* https://seekingalpha.com/
* https://www.zacks.com/
* https://finance.yahoo.com/

In [1]:
# installation libraries
! pip install yfinance --upgrade --no-cache-dir &> /dev/null
! pip install beautifulsoup4 &> /dev/null
! pip install cloudscraper &> /dev/null

In [2]:
# import libraries
import yfinance as yf
import pandas as pd
from pandas.core.internals.base import DataManager
from bs4 import BeautifulSoup
import cloudscraper
import plotly.graph_objects as go
import plotly.express as px

scraper = cloudscraper.create_scraper()

# disk connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**List with the data we are interested in**

In [15]:
# columns = ['symbol','currentPrice','recommendationKey', 'recommendationMean', 'targetLowPrice', 'targetLowPrice [%]',
#          'targetMeanPrice', 'targetMeanPrice [%]', 'targetMedianPrice', 'targetMedianPrice [%]', 'targetHighPrice', 'targetHighPrice [%]',
#          'P/E', 'Forward P/E', 'EPS past 5Y', 'EPS (ttm)', 'forwardEps', 'EPS next Q', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
#           'ROA', 'ROE', 'ROI', 'currentRatio', 'quickRatio']

columns = ['symbol','currentPrice','recommendationKey', 'recommendationMean', 'targetLowPrice [%]',
           'targetMeanPrice [%]', 'targetMedianPrice [%]', 'targetHighPrice [%]', 'numberOfAnalystOpinions',
           'P/E', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
           'ROA', 'ROE', 'ROI', 'currentRatio', 'quickRatio']

**Lists with tickers we are interested in**

In [16]:
finansowa_edu = [ 'NFLX','MSFT','AAPL', 'NVDA','V','JNJ','AZN','SAP', 'RGEN','PG','AMT','GOOGL','AMZN']
Trela = ['TSM','INTU','NOW','PTC','ALGN','AMZN','QCOM', 'MA','ASML','MPWR','NVDA', 'AMD']
Kupione = ['EXAS', 'META', 'NIO', 'PDD', 'CRWD', 'MKSI', 'MDT', 'TSM', 'NFLX', 'MSFT', 'AAPL', 'V', 'BABA', 'TSM']
DNA = ['ROKU', 'TWLO', 'SE', 'DIS', 'UUUU', 'DNN']

**Tickers from seekingalpha if we want**

In [17]:
# data from a file on disk
seekingalpha = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TopGrowth.csv')
alpha = list(seekingalpha.Symbol)

**Selection of a list of tickers for analysis**

In [18]:
Tickers = alpha
output_name = 'alpha'

**Downloading data from finviz and finance.yahoo**

In [19]:
asset = pd.DataFrame(columns=columns)

for tic in Tickers: 

  url = 'https://finviz.com/quote.ashx?t=' + tic
  page = scraper.get(url)
  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('tr', class_='table-dark-row')
  
  if table == []:
    continue

  names = []
  value = []

  # Downloading data from finviz (website source)
  for row in table:
    for name in row.find_all('td', class_='snapshot-td2-cp'):
      names.append(name.text)

    for name in row.find_all('td', class_='snapshot-td2'):
      if name.text == '-':
        v = 0
      else:
        v = name.text
      value.append(v)

  # Data connection in dictionary
  finviz = dict(zip(names, value))

  # Downloading data from finance.yahoo API
  data = yf.Ticker(tic).info

  # Data connection finance.yahoo + finviz
  data.update(finviz)

# Conversion to percentages
  for_percentage = {'targetLowPrice [%]':'targetLowPrice', 'targetMeanPrice [%]':'targetMeanPrice',
                    'targetMedianPrice [%]': 'targetMedianPrice', 'targetHighPrice [%]':'targetHighPrice'}

  for col in for_percentage:
    try:
      data[col] = round(100 * float(data[for_percentage[col]])/float(data['currentPrice'])-100)
    except:
      data[col] = '-'

  row={}
  for col in columns:
    row[col] = data[col]

  asset = asset.append(row, ignore_index=True)


# Columns that need to be reformatted to number
colum = ['EPS next Y', 'EPS next 5Y', 'Profit Margin', 'ROA', 'ROE', 'ROI']

for col in colum:
  asset[col] = asset[col].str.replace("%", "").astype(float)

asset.set_index('symbol', inplace=True)

**Downloading data from Zacks**

In [20]:
zacks = []

for tic in Tickers:
  url = 'https://www.zacks.com/stock/quote/' + tic
  page = scraper.get(url)
  page_html = BeautifulSoup(page.content, 'html.parser')
  table = page_html.find_all('p', class_="rank_view")
  try:
    zac = list(table[0].get_text().replace(" ", ""))[1]
  except IndexError:
    zacks.append(0)
    continue

  zacks.append(zac)

**Table formatting for future connection (if you use seekingalpha)**

In [10]:
seekingalpha['zacks'] = zacks
seekingalpha.drop(columns='Rank', inplace=True)
seekingalpha.set_index('Symbol', inplace=True)
asset.rename_axis("Symbol", inplace=True)

#Table joins
asset = pd.merge(left=seekingalpha, right=asset, on='Symbol')

**Saving data to disk**

In [ ]:
asset.to_csv('/content/drive/MyDrive/Colab Notebooks/'+ output_name +'.csv')

**Data visualization**

In [ ]:
# chart titles
title = ['Recommendation mean', 'Target low price [%]',
           'Target mean price [%]', 'Target median price [%]', 'Target high price [%]',
           'P/E', 'EPS next Y', 'EPS next 5Y', 'Profit Margin',
           'ROA', 'ROE', 'ROI', 'Current ratio', 'Quick ratio']

col_tite = dict(zip(columns[3:], title))

for statistics in col_tite:
  fig = px.bar(asset, x=asset.index, y=statistics, title=col_tite[statistics])
  fig.show()

In [ ]:
def chart(symbol):

  tic = yf.Ticker(symbol)
  data = tic.history(period = "ytd")
  data['targetLowPrice'] = asset.loc[symbol, 'targetLowPrice']
  data['targetMeanPrice'] = asset.loc[symbol, 'targetMeanPrice']
  data['targetHighPrice'] = asset.loc[symbol, 'targetHighPrice']

  fig = go.Figure()

  fig.add_trace(go.Scatter(x=data.index, y=data.Close,
                             mode='lines',
                             name='Close'))
  
  fig.add_trace(go.Scatter(x=data.index, y=data['targetLowPrice'],
                             mode='lines',
                             name=asset.loc[symbol, 'targetLowPrice [%]']))
  
  fig.add_trace(go.Scatter(x=data.index, y=data['targetMeanPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetMeanPrice [%]']))

  fig.add_trace(go.Scatter(x=data.index, y=data['targetHighPrice'],
                           mode='lines',
                           name=asset.loc[symbol, 'targetHighPrice [%]']))
  fig.show()

In [ ]:
chart('BABA')